In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbtpro as vbt
import warnings
warnings.filterwarnings('ignore')

In [2]:
vbt.settings.plotting["layout"]["template"] = "vbt_dark"
vbt.settings.plotting["layout"]["width"] = 800
vbt.settings.plotting['layout']['height'] = 350
vbt.settings.wrapping["freq"] = "1m"
# vbt.settings.portfolio['size_granularity'] = 0.001
vbt.settings.portfolio['init_cash'] = 10000

In [39]:
# Read in the 1 minute data from my HDF5 file
data = vbt.Data.from_hdf('../data/data.hdf5')
# Make it a binance data object so we can use resampling dict and other binance specific functions
data = vbt.BinanceData.from_data(data.get())



Run talib features on the dataframe with various lookback window params as well as resampled timeframes

In [73]:
sma_ranges = data.resample('1h').loc['2021'].run("SMA", timeperiod=[15, 30, 45], timeframe=["4h", "8h", "1d"], skipna=True)
sma_ranges.real.vbt.ts_heatmap().show()

In [61]:
sma_ranges.real

sma_timeperiod,15,30,45
sma_timeframe,4h,8h,1d
Open time,,,
2021-01-01 00:00:00+00:00,NaN,NaN,NaN
2021-01-01 01:00:00+00:00,NaN,NaN,NaN
2021-01-01 02:00:00+00:00,NaN,NaN,NaN
2021-01-01 03:00:00+00:00,NaN,NaN,NaN
2021-01-01 04:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2021-12-31 19:00:00+00:00,47216.781333,49106.353667,51819.223333
2021-12-31 20:00:00+00:00,47115.380000,49106.353667,51819.223333


Run entire list of talib features

In [67]:
features = data.resample('1h').run("talib", mavp=vbt.run_arg_dict(periods=13))

In [68]:
features.shape

(20780, 175)

Let's have a look at the correlations between features

In [70]:
# Compute correlation matrix
correlation_matrix = features.corr()
correlation_matrix.vbt.heatmap().show()


Add all the world quant 101 alphas 

In [88]:
alpha_number = 1 # 1 through 101
data.run(f"wqa101_{alpha_number}").out

Open time
2021-01-01 00:00:00+00:00    NaN
2021-01-01 00:01:00+00:00    NaN
2021-01-01 00:02:00+00:00    NaN
2021-01-01 00:03:00+00:00    NaN
2021-01-01 00:04:00+00:00    NaN
                            ... 
2023-05-16 19:52:00+00:00    0.5
2023-05-16 19:53:00+00:00    0.5
2023-05-16 19:54:00+00:00    0.5
2023-05-16 19:55:00+00:00    0.5
2023-05-16 19:56:00+00:00    0.5
Name: Close, Length: 1245724, dtype: float64

For more info on the alphas see here https://vectorbt.pro/pvt_6739a1da/api/indicators/expr/#vectorbtpro.indicators.expr.wqa101_expr_config and here https://vectorbt.pro/pvt_6739a1da/documentation/indicators/parsers/#worldquants-alphas  

In [103]:
# just iterate over them one at a time
wqa_features = data.resample('1h').run(["wqa101_%d" % i for i in range(1, 102)])

In [104]:
# Create a correlation matrix of the features
correlation_matrix = wqa_features.corr()
correlation_matrix.vbt.heatmap().show()